## Instalações

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#!pip install pandas
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)
plt.style.use("ggplot")


from seqeval.metrics import f1_score, classification_report, precision_score, recall_score, accuracy_score
#! pip install plot_keras_history
#from plot_keras_history import plot_history

#!pip install scikit-learn==0.22.2

#!pip install sklearn_crfsuite
#from sklearn_crfsuite import CRF

#!python -m spacy download pt_core_news_sm

In [ ]:
from seqeval.scheme import IOB2

## Dados

In [ ]:
import pandas as pd

df = pd.read_csv('./CSVs/V2/DODFCorpus_contratos_licitacoes_v2.csv', dtype=str)
df = df.drop(['Unnamed: 0','Unnamed: 0.1'], axis =1)

In [ ]:
##Insere um espaço entre as entidade e insere um espaco apos :  : -> Processo:0...
import re
def correct_space_before_numeric_entities(string):
    result = re.sub(r'([A-Za-z]:)[0-9]', r'\1 ', string)
    result = result.replace("\n", " ")
    return result

In [ ]:
df['texto']= df['texto'].map(correct_space_before_numeric_entities)

In [ ]:
df.tipo_rel.unique()

In [ ]:
#Concatena o id_ato com o id_dodf
df['id_ato'] = df['id_dodf'] + '-' + df['id_rel']
#data

In [ ]:
#Lista o tipo de relatório
result =df.groupby('tipo_rel')['id_ato'].nunique()
print(result.sort_values(ascending=False))

In [ ]:
tipo_ato = df.query("tipo_rel == 'REL_EXTRATO_CONVENIO'").reset_index(drop=True)

In [ ]:
#Lista as entidades do tipo de ato
result = tipo_ato.groupby('tipo_ent')['id_ato'].nunique()
print(result)
#print(result.sort_values(ascending=False))

In [ ]:
act_name = 'EXTRATO_CONVENIO'

In [ ]:
max_length = 400

## Geração do IOB

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
from mimetypes import init
import re
import spacy
from spacy.lang.pt.examples import sentences 
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex
from spacy.tokens import DocBin

class spacy_tokenizer():
    def __init__(self):
        #self.nlp = spacy.load('pt_core_news_sm', disable=["ner", "lemmatizer"])
        self.nlp = spacy.load('pt_core_news_sm', disable = ['parser','ner'])
        

    def tokenize(self, texto):
                
        doc = self.nlp(texto)
        return[t.text.strip() for t in doc if t.text.strip()]

In [ ]:
class iob_transformer():
    
    def __init__(self, coluna_id_ato: str, coluna_texto_entidade: str,
                 coluna_tipo_entidade: str, keep_punctuation: bool = False,
                 return_df: bool = False, tokenizer_tipo: str = "Função"):
        self.coluna_id_ato = coluna_id_ato
        self.coluna_texto_entidade = coluna_texto_entidade
        self.coluna_tipo_entidade = coluna_tipo_entidade
        self.tokenizer_tipo = tokenizer_tipo
        if not keep_punctuation: #False
            #self.tokenizer = RegexpTokenizer('\w+')
            self.tokenizer = spacy_tokenizer()
        else:
            self.tokenizer = False
        self.return_df = return_df

    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def gera_listas_atos_iobs(self, df):
        
        def _inclui_tags_vazias(texto_iob):
            texto_ato_iob = texto_iob.copy()
            for idx, token in enumerate(texto_ato_iob):
                if token[0:2] == 'B-':# in token:
                    pass
                elif token[0:2] == 'I-':# in token:
                    pass
                else:
                    texto_ato_iob[idx] = 'O'
            
            return texto_ato_iob
        
        def _constroi_iob(texto_ent, tipo_ent):
            if self.tokenizer:
                texto_ent_tok = self.tokenizer.tokenize(texto_ent)
                
                
            else:
                texto_ent_tok = word_tokenize(texto_ent)
            iob_entidade = []
            for index_token, token in enumerate(texto_ent_tok):
                # primeiro token?
                if index_token == 0:
                    palavra = 'B-'+ tipo_ent
                    iob_entidade.append(palavra)
                # é o segundo token?
                else:
                    palavra = 'I-'+ tipo_ent
                    iob_entidade.append(palavra)
            # salva tupla contendo texto tokenizado e iob correspondente
            if self.tokenizer:
                tup_entidade = (self.tokenizer.tokenize(texto_ent), iob_entidade)
            
            else:
                tup_entidade = (word_tokenize(texto_ent), iob_entidade)
            return tup_entidade
        
        def _match_iob_texto_ato(texto_entidade_tok, iob_ato):
            texto_ato_iob = texto_entidade_tok.copy()
            #print(iob_ato)
            for tupla in iob_ato:
                 # checa se o texto de referência existe
                if tupla[0]:
                    for i in range(len(texto_entidade_tok)):
                        # checa se a tag existe
                        if tupla[0][0]:
                            # match primeiro token
                            if texto_entidade_tok[i] == tupla[0][0]:
                                # a sequência de tokens de texto_entidade_token na
                                # posição encontrada é igual aos tokens da entidade?
                                if texto_entidade_tok[i:i+len(tupla[0])] == tupla[0]:
                                    texto_ato_iob[i:i+len(tupla[0])] = tupla[1]
            
            return texto_ato_iob

        atos = []
        lista_labels = []
        id_atos = set()
        for row in df.iterrows():
            id_ato = df.iloc[row[0]][self.coluna_id_ato]
            texto_ato = []
            texto_ato_iob = []
            if id_ato not in id_atos:
                id_atos.add(id_ato)
                lista_ids = list(df.query(f'{self.coluna_id_ato} == "{id_ato}"').index)
                
                iob_ato = []
                # todas as anotações que não são o ato inteiro
                for index in lista_ids:
                    texto_entidade = df.iloc[index][self.coluna_texto_entidade]
                    tipo_entidade = df.iloc[index][self.coluna_tipo_entidade]
                    #Lucelia if isinstance(self.tokenizer, RegexpTokenizer):
                    if self.tokenizer_tipo == "Função":
                        texto_entidade_tok = self.tokenizer.tokenize(texto_entidade)

                    else:
                        texto_entidade_tok = word_tokenize(texto_entidade)
                    if not tipo_entidade.isupper():
                        tup_entidade = _constroi_iob(texto_entidade, tipo_entidade)
                        iob_ato.append(tup_entidade)
                # anotação do ato inteiro
                for index in lista_ids:
                    texto_entidade = df.iloc[index][self.coluna_texto_entidade]
                    tipo_entidade = df.iloc[index][self.coluna_tipo_entidade]
                    
                    if self.tokenizer_tipo == "Função":
                        texto_entidade_tok = self.tokenizer.tokenize(texto_entidade)
                    else:
                        texto_entidade_tok = word_tokenize(texto_entidade)
                    if tipo_entidade.isupper():
                        texto_ato = texto_entidade_tok
                        texto_ato_iob = _match_iob_texto_ato(texto_entidade_tok, iob_ato)
                texto_ato_iob = _inclui_tags_vazias(texto_ato_iob)
                atos.append(texto_ato)
                lista_labels.append(texto_ato_iob)
        
        return atos, lista_labels

    def create_iob_df(self, atos, lista_labels):
        rows_list = []
        dict1 = {
                'Sentence_idx': -1,
                'Word': 'UNK',
                'Tag': 'O'
            }
        rows_list.append(dict1)
        id_ato = 0
        for ato, labels in zip(atos, lista_labels):
            for word, label in zip(ato, labels):
                dict1 = {
                    'Sentence_idx': id_ato,
                    'Word': word,
                    'Tag': label
                }
                rows_list.append(dict1)
                
            id_ato += 1
        new_df = pd.DataFrame(rows_list)

        return new_df    
    
    def transform(self, df, **transform_params):
        dataframe = df.copy()
        dataframe = dataframe.reset_index(drop=True)
        atos, lista_labels = self.gera_listas_atos_iobs(dataframe)
        if self.return_df:
            iob_df = self.create_iob_df(atos, lista_labels)
            return iob_df
        else:
            return atos, lista_labels

In [ ]:
iob = iob_transformer('id_ato','texto','tipo_ent', keep_punctuation=False, return_df=False)

In [ ]:
acts, labels = iob.transform(tipo_ato)

## Ajustes das labels e dicionários

In [ ]:
def remove_wrong_tags(label_list):
      for label in label_list:
    for idx,w in enumerate(label):
      if w in ['B-11','B-12','B-50', 'B-60', 'I-2']:
        label[idx] = 'O'

In [ ]:
remove_wrong_tags(labels)

In [ ]:
for act in acts:
    for word in act:
        words.add(word)
#convertendo o set em uma lista
words = list(words)

words.append("ENDPAD")
words.append("UNK")

words_amt = len(words)

In [ ]:
tags = set()

for label in labels:
    for tag in label:
        tags.add(tag)
tags = list(tags)
tags_amt = len(tags)

## Transformação dos dados

In [ ]:
vocab = {}

for i in range(0,len(acts)):
    for word in acts[i]:
        if word.lower() not in vocab:
            vocab[word.lower()] = 1
        else:
            vocab[word.lower()]+=1

In [ ]:
def word2features(sent, i):
    word = sent[i]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),        
    }
    if i > 0:
        word1 = sent[i-1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True
                
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [ ]:
for i in range(0,5):
  inputs = [sent2features(s) for s in acts]

In [ ]:
targets = labels

## K-fold

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state = 42)

## CRF

In [ ]:
import os, pickle
acc = []
loss = []
f1 = []
reports = []
fold = 0

for train, test in kfold.split(inputs, targets):

  x_train = np.array(inputs)[train.astype(int)]
  x_test = np.array(inputs)[test.astype(int)]

  y_train = np.array(targets)[train.astype(int)]
  y_test = np.array(targets)[test.astype(int)]
 
  with open(os.path.join('./CSVs/V2/split/CISTI/WS/teste_ws/'+act_name+'_'+str(fold)+'_x_test.txt'), 'wb') as file:
        pickle.dump(x_test, file)
  with open(os.path.join('./CSVs/V2/split/CISTI/WS/teste_ws/'+act_name+'_'+str(fold)+'_y_test.txt'), 'wb') as file:    
        pickle.dump(y_test, file)


#act type	 model	 c1	 c2
#aviso de licitação	 crf	 0.0629	 0.0510
#extrato de contrato/convênio	 crf	 0.1094	 0.0230
#aviso de suspensão de licitação	 crf	461	 0.05

#extrato de aditamento contratual	 crf	 0.0773	 0.1447
#aviso de revogação/anulação de licitação	 crf	 0.1181	 0.0069

  #valores c1:0.461, c2:0.039
  crf = CRF(
  algorithm='lbfgs', 
  c1=	0.1094, 
  c2= 0.0230, 
  max_iterations=70, 
  all_possible_transitions=True, verbose=1)

  crf.fit(X=x_train, y=y_train)

  y_pred = crf.predict(x_test)

  f1.append(f1_score(y_test, y_pred))

  reports.append(classification_report(y_test,y_pred))

  acc.append(accuracy_score(y_test,y_pred))

  r = classification_report(y_test,y_pred, output_dict=True, mode='strict', scheme=IOB2)

  name = act_name + '_f' + str(fold) +".npy"
  np.save('./Results/V2/CISTI/27_06/CRF/'+name, r)

  fold = fold + 1
  reports.append(r)

In [ ]:
print(reports[0])

In [ ]:
f1

In [ ]:
np.mean(f1)

In [ ]:
np.std(f1)